In [27]:
import pandas as pd
import nltk
from nltk.collocations import *
from scipy.stats import spearmanr

In [7]:
df = pd.read_csv('court-V-N.csv', header=None) # corpus
rows = list(map(list, df.values)) # list of rows in corpus
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(rows)
len([item for sublist in rows for item in sublist]) # количесство токенов

5100

Пользуясь словарями, интуицией и т.п. составляете золотой стандарт – ранжированный топ 10 коллокаций

In [21]:
golden_stand = [('ПРИНЯТЬ', 'РЕШЕНИЕ'),('СУД', 'ПРИЗНАТЬ'),('СУД', 'ВЫНЕСТИ'),('УДОВЛЕТВОРИТЬ', 'ИСК'),
                 ('УДОВЛЕТВОРИТЬ', 'ХОДАТАЙСТВО'),('ПРИЗНАТЬ', 'ВИНОВНАЯ'),('ВЫНЕСТИ', 'РЕШЕНИЕ'),
                 ('СУД', 'ОТКАЗАТЬ'),('СУД', 'ПРИГОВОРИТЬ'),('НАЛОЖИТЬ', 'АРЕСТ')]

Выделить наиболее типичные для суда действия (словосочетания типа *принять решение*), применив два различных метода.
Я взяла likelihood_ratio и raw_freq

In [11]:
top_lr = finder.nbest(bigram_measures.likelihood_ratio, 10)
top_lr

[('СУД ', 'ПРИЗНАТЬ '),
 ('ВЫДАТЬ ', 'САНКЦИЯ '),
 ('СУД ', 'УДОВЛЕТВОРИТЬ '),
 ('УДОВЛЕТВОРИТЬ ', 'ИСК '),
 ('УДОВЛЕТВОРИТЬ ', 'ХОДАТАЙСТВО '),
 ('ПРИНЯТЬ ', 'РЕШЕНИЕ '),
 ('САНКЦИЯ ', 'АРЕСТ '),
 ('МЕРА ', 'ПРЕСЕЧЕНИЕ '),
 ('НАЛОЖИТЬ ', 'АРЕСТ '),
 ('СУД ', 'ВЫНЕСТИ ')]

**(а) какие коллокации, полученные методом lr, не входят в Ваш золотой стандарт:**<br>
не вошли 4 коллокации: ('ВЫДАТЬ ', 'САНКЦИЯ '), ('СУД ', 'УДОВЛЕТВОРИТЬ '), ('САНКЦИЯ ', 'АРЕСТ '), ('МЕРА ', 'ПРЕСЕЧЕНИЕ ')<br>
**(б) какие попали ошибочно (предложите объяснение - откуда взялась такая ошибка, что можно сделать, чтобы ее устранить)**<br>
мне кажется, все коллокации хорошие<br>
**(в) какие у Вас не попоали в ЗС, но вы думаете, что это полезные коллокации**<br>
все 4, которые не попали в ЗС

In [14]:
top_rf = finder.nbest(bigram_measures.raw_freq, 10)
top_rf

[('СУД ', 'ПРИЗНАТЬ '),
 ('СУД ', 'УДОВЛЕТВОРИТЬ '),
 ('СУД ', 'ВЫНЕСТИ '),
 ('СУД ', 'ПРИНЯТЬ '),
 ('СУД ', 'ОТКАЗАТЬ '),
 ('СУД ', 'ПРИГОВОРИТЬ '),
 ('СУД ', 'РАССМОТРЕТЬ '),
 ('УДОВЛЕТВОРИТЬ ', 'ИСК '),
 ('СУД ', 'ОТКЛОНИТЬ '),
 ('ПРИНЯТЬ ', 'РЕШЕНИЕ ')]

**(а) какие коллокации, полученные методом rf, не входят в Ваш золотой стандарт:**<br>
не вошли тоже 4 коллокации: ('СУД ', 'УДОВЛЕТВОРИТЬ '), ('СУД ', 'ПРИНЯТЬ '), ('СУД ', 'РАССМОТРЕТЬ '), ('СУД ', 'ОТКЛОНИТЬ ')<br>
**(б) какие попали ошибочно (предложите объяснение - откуда взялась такая ошибка, что можно сделать, чтобы ее устранить)**<br>
мне кажется, все коллокации хорошие<br>
**(в) какие у Вас не попоали в ЗС, но вы думаете, что это полезные коллокации**<br>
точно ('СУД ', 'УДОВЛЕТВОРИТЬ '), а над остальными стоит подумать

In [26]:
list_with_scores_lr = list(finder.score_ngrams(bigram_measures.likelihood_ratio))
list_with_scores_rf = list(finder.score_ngrams(bigram_measures.raw_freq))
list_no_scores_lr = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_lr]
list_no_scores_rf = [(x[0][0].strip(' '), x[0][1].strip(' ')) for x in list_with_scores_rf]
ranks_golden = list(range(0,10))
ranks_lr = [list_no_scores_lr.index(i) for i in golden_stand]
ranks_rf = [list_no_scores_rf.index(i) for i in golden_stand]
print('gold', ranks_golden)
print('lr', ranks_lr)
print('rf', ranks_rf)

gold [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
lr [5, 0, 9, 3, 4, 12, 16, 15, 24, 8]
rf [9, 0, 2, 7, 10, 20, 14, 4, 5, 25]


In [42]:
# Spearman

print('lr gs', spearmanr(ranks_lr,ranks_golden))
print('rf gs', spearmanr(ranks_rf,ranks_golden))
print('lr rf', spearmanr(ranks_lr,ranks_rf))

lr gs SpearmanrResult(correlation=0.6484848484848483, pvalue=0.04254012768448945)
rf gs SpearmanrResult(correlation=0.4424242424242424, pvalue=0.20042268671194224)
lr rf SpearmanrResult(correlation=0.17575757575757575, pvalue=0.6271883447764844)
